In [6]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../")

from src.enzyme_hackathon.utils import (one_hot_dict, one_hot_encode, one_hot_encode_screening_data,
                                        one_hot_encode_sequences)
from src.enzyme_hackathon.utils import AA_LABELS
from src.enzyme_hackathon.utils import (create_fc_model, predict, generate_variant, r_squared, nan_mse)
import keras
import tensorflow as tf

In [7]:
?tf.save

Object `tf.save` not found.


In [8]:
keras.models.load_model("../models/FCNN_50_ADAM.hfd5")

ValueError: Unknown loss function:nan_mse